In [1]:
import numpy as np
import random
from fractions import Fraction
import re
import time

In [2]:
class GeneticEquationSolver:
    def __init__(self, pop_size=100, generations=1000, mutation_rate=0.1, elite_size=10,
                 var_range=(-10, 10), tournament_size=3):
        """        
        pop_size: Population size
        generations: Number of generations
        mutation_rate: Mutation rate
        elite_size: Number of elites to directly transfer to next generation
        var_range: Range of variable values
        tournament_size: Tournament size for parent selection
        """
        self.pop_size = pop_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.var_range = var_range
        self.tournament_size = tournament_size
        
    def fitness_function(self, chromosome, equations):
        """        
        chromosome: Chromosome (set of variable values)
        equations: Equation functions
        :return: Inverse of total error (higher value means less error)
        """
        total_error = 0
        for eq in equations:
            try:
                # Calculate error for each equation - handle potential errors
                result = eq(*chromosome)
                total_error += abs(result)
            except ZeroDivisionError:
                # Penalize solutions that cause division by zero
                total_error += 1e6  # Large penalty
            except (ValueError, OverflowError, TypeError):
                # Penalize other errors
                total_error += 1e5
        
        # To avoid division by zero
        if total_error < 1e-10:
            return float('inf')
        return 1.0 / total_error
    
    def initialize_population(self, var_count):
        """        
        var_count: Number of variables
        :return: Initial population
        """
        population = []
        for _ in range(self.pop_size):
            # For each variable, select a random value in the specified range
            chromosome = []
            for _ in range(var_count):
                # Value can be fraction
                if random.random() < 0.5:  # 50% chance of integer
                    gene = random.randint(self.var_range[0], self.var_range[1])
                    # Avoid zero if possible (causes division by zero)
                    if gene == 0:
                        gene = random.choice([-1, 1])
                else:  # 50% chance of fraction
                    numerator = random.randint(self.var_range[0] * 10, self.var_range[1] * 10)
                    denominator = random.randint(1, 10)
                    gene = numerator / denominator / 10
                    # Avoid zero if possible
                    if abs(gene) < 1e-10:
                        gene = random.choice([-0.1, 0.1])
                chromosome.append(gene)
            population.append(chromosome)
        return population
    
    def evaluate_population(self, population, equations):
        """
        Evaluate population by calculating fitness of each chromosome
        
        population: Population
        equations: Equation functions
        :return: List of (chromosome, fitness)
        """
        fitness_scores = []
        for chromosome in population:
            fitness = self.fitness_function(chromosome, equations)
            fitness_scores.append((chromosome, fitness))
        return sorted(fitness_scores, key=lambda x: x[1], reverse=True)
    
    def selection(self, ranked_population):
        """        
        ranked_population: Population ranked by fitness
        :return: Selected chromosome
        """
        tournament = random.sample(ranked_population, self.tournament_size)
        return max(tournament, key=lambda x: x[1])[0]
    
    def crossover(self, parent1, parent2):
        """        
        parent1: First parent
        parent2: Second parent
        :return: Created child
        """
        child = []
        # Uniform crossover: each gene has 50% chance from each parent
        for gene1, gene2 in zip(parent1, parent2):
            if random.random() < 0.5:
                child.append(gene1)
            else:
                child.append(gene2)
        return child
    
    def mutation(self, chromosome):
        """        
        chromosome: Original chromosome
        :return: Mutated chromosome
        """
        mutated = chromosome.copy()
        for i in range(len(mutated)):
            # Each gene mutates with probability mutation_rate
            if random.random() < self.mutation_rate:
                if random.random() < 0.7:  # 70% chance of a new random value
                    if random.random() < 0.5:  # 50% chance of integer
                        mutated[i] = random.randint(self.var_range[0], self.var_range[1])
                        # Avoid zero if possible
                        if mutated[i] == 0:
                            mutated[i] = random.choice([-1, 1])
                    else:  # 50% chance of fraction
                        numerator = random.randint(self.var_range[0] * 10, self.var_range[1] * 10)
                        denominator = random.randint(1, 10)
                        mutated[i] = numerator / denominator / 10
                        # Avoid zero if possible
                        if abs(mutated[i]) < 1e-10:
                            mutated[i] = random.choice([-0.1, 0.1])
                else:  # 30% chance of a small change
                    # Small change to current value
                    mutated[i] += random.uniform(-1, 1)
                    # Avoid zero if possible
                    if abs(mutated[i]) < 1e-10:
                        mutated[i] = random.choice([-0.1, 0.1])
        return mutated
    
    def next_generation(self, current_ranked_population, equations):
        """        
        current_ranked_population: Current sorted population
        equations: Equation functions
        :return: Next generation population
        """
        next_population = []
        
        # Transfer elites to next generation
        elites = [x[0] for x in current_ranked_population[:self.elite_size]]
        next_population.extend(elites)
        
        # Complete population with new children
        while len(next_population) < self.pop_size:
            parent1 = self.selection(current_ranked_population)
            parent2 = self.selection(current_ranked_population)
            
            child = self.crossover(parent1, parent2)
            child = self.mutation(child)
            
            next_population.append(child)
        
        return next_population
    
    def solve(self, equations, var_count):
        """        
        equations: Equation functions
        var_count: Number of variables
        :return: Best solution found
        """
        # Create initial population
        population = self.initialize_population(var_count)
        
        best_solution = None
        best_fitness = -float('inf')
        
        # Iterate for specified number of generations
        for generation in range(self.generations):
            # Evaluate population
            ranked_population = self.evaluate_population(population, equations)
            
            # Check best solution in this generation
            current_best = ranked_population[0]
            if current_best[1] > best_fitness:
                best_solution = current_best[0]
                best_fitness = current_best[1]
                
                # Check if exact solution found
                if best_fitness > 1e6:  # Large value indicates very small error
                    print(f"Exact solution found in generation {generation}.")
                    break
            
            # Show progress every 100 generations
            if generation % 100 == 0:
                error = 1.0 / best_fitness if best_fitness not in [float('inf'), 0] else 0
                print(f"Generation {generation}: Best error = {error}")
            
            # Create next generation
            population = self.next_generation(ranked_population, equations)
        
        # Convert solution to simplified fractions
        solution_fractions = self.approximate_fractions(best_solution)
        
        # Final check of solution
        error = 1.0 / best_fitness if best_fitness not in [float('inf'), 0] else 0
        print(f"Best solution: {solution_fractions}")
        print(f"Final error: {error}")
        
        return solution_fractions
    
    def approximate_fractions(self, solution, max_denominator=100):
        """
        Convert decimal numbers to approximate fractions
        
        solution: Solution as decimal numbers
        max_denominator: Maximum denominator
        :return: Solution as simplified fractions
        """
        result = []
        for value in solution:
            fraction = Fraction(value).limit_denominator(max_denominator)
            result.append(fraction)
        return result

# Helper functions for converting equations to Python functions

In [3]:
def parse_linear_equation_2var(equation_str):
    """
    Convert a linear equation with two variables to a Python function
    Example: "x + 2y = 4" to a function that calculates the equation error
    """
    # Remove spaces
    equation_str = equation_str.replace(" ", "")
    
    # Split left and right sides
    left_side, right_side = equation_str.split("=")
    
    # Find coefficients of x and y
    coef_x, coef_y = 0, 0
    
    # Check x coefficient
    x_terms = re.findall(r'([+-]?\d*)[xX]', left_side)
    for term in x_terms:
        if term in ['+', '']:
            coef_x += 1
        elif term == '-':
            coef_x -= 1
        else:
            coef_x += int(term)
    
    # Check y coefficient
    y_terms = re.findall(r'([+-]?\d*)[yY]', left_side)
    for term in y_terms:
        if term in ['+', '']:
            coef_y += 1
        elif term == '-':
            coef_y -= 1
        else:
            coef_y += int(term)
    
    # Convert right side to number
    right_side_val = float(right_side)
    
    # Create error function
    def equation_error(x, y):
        return abs(coef_x * x + coef_y * y - right_side_val)
    
    return equation_error

def parse_advanced_equation(equation_str, var_names):
    """
    Convert a complex equation to a Python function using eval
    
    :param equation_str: Equation string
    :param var_names: Variable names (e.g., ['x', 'y', 'z'])
    :return: Function that calculates the equation error
    """
    # Split left and right sides
    sides = equation_str.replace(" ", "").split("=")
    left_side = sides[0]
    right_side = sides[1]
    
    # Create error function with exception handling
    def equation_error(*vars):
        # Create dictionary of variables and their values
        var_dict = {name: value for name, value in zip(var_names, vars)}
        
        try:
            # Calculate both sides of the equation
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            # Error is the absolute difference
            return abs(left_val - right_val)
        except ZeroDivisionError:
            # Return a large error if division by zero occurs
            return 1e6
        except (ValueError, OverflowError, TypeError):
            # Return a large error for other errors
            return 1e5
    
    return equation_error

def parse_equations_system(equations_str, var_names):
    """
    Convert a system of equations to a list of Python functions
    
    :param equations_str: List of equation strings
    :param var_names: Variable names
    :return: List of error functions
    """
    equation_functions = []
    for eq_str in equations_str:
        equation_functions.append(parse_advanced_equation(eq_str, var_names))
    return equation_functions

def print_fraction(frac):
    """Display fraction in readable format"""
    if frac.denominator == 1:
        return str(frac.numerator)
    else:
        return f"{frac.numerator}/{frac.denominator}"

In [4]:
def solve_equations(equations, var_names):
    """Equation system solver using genetic algorithm"""
    # Convert equations to functions
    equations_funcs = parse_equations_system(equations, var_names)
    
    # Create equation solver with genetic algorithm
    solver = GeneticEquationSolver(
        pop_size=200, 
        generations=1000, 
        mutation_rate=0.1, 
        elite_size=20,
        var_range=(-20, 20)
    )
    
    # Solve equations
    solution = solver.solve(equations_funcs, len(var_names))
    
    # Display results
    print("\nFinal solution:")
    for var, val in zip(var_names, solution):
        print(f"{var} = {print_fraction(val)}")
    
    # Verify solution in all equations
    print("\nVerifying solution in equations:")
    for i, eq_str in enumerate(equations):
        try:
            # Create dictionary of variables and their values
            var_dict = {name: float(value) for name, value in zip(var_names, solution)}
            
            # Calculate both sides of the equation
            sides = eq_str.replace(" ", "").split("=")
            left_side = sides[0]
            right_side = sides[1]
            
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            print(f"Equation {i+1}: {left_val} ≈ {right_val}, error: {abs(left_val - right_val)}")
        except Exception as e:
            print(f"Equation {i+1}: Error in verification - {str(e)}")

def solve_example_2var():
    """Solve example with two variables"""
    print("\n=== Solving equation system with 2 variables ===")
    equations = [
        "x + 2*y = 4",
        "4*x + 4*y = 12"
    ]
    solve_equations(equations, ['x', 'y'])

def solve_example_3var():
    """Solve example with three variables"""
    print("\n=== Solving equation system with 3 variables ===")
    equations = [
        "6*x - 2*y + 8*z = 20",
        "y + 8*x * z = -1",
        "2*z * (6/x) + (3/2)*y = 6"
    ]
    solve_equations(equations, ['x', 'y', 'z'])

def solve_example_4var():
    """Solve example with four variables"""
    print("\n=== Solving equation system with 4 variables ===")
    equations = [
        "(1/15)*x - 2*y - 15*z - (4/5)*t = 3",
        "-(5/2)*x - (9/4)*y + 12*z - t = 17",
        "-13*x + (3/10)*y - 6*z - (2/5)*t = 17",
        "(1/2)*x + 2*y + (7/4)*z + (4/3)*t = -9"
    ]
    solve_equations(equations, ['x', 'y', 'z', 't'])

def solve_custom_equations(equations, var_names):
    """Solve custom equation system"""
    print("\n=== Solving custom equation system ===")
    solve_equations(equations, var_names)

# Run the examples
# solve_example_2var()
# solve_example_3var()
solve_example_4var()


=== Solving equation system with 4 variables ===
Generation 0: Best error = 17.06590277777778
Generation 100: Best error = 0.6512449574915236
Generation 200: Best error = 0.5716052271917018
Generation 300: Best error = 0.5708675045285765
Generation 400: Best error = 0.5708129613501671
Generation 500: Best error = 0.5708129613501671
Generation 600: Best error = 0.5708129613501671
Generation 700: Best error = 0.5705455146915956
Generation 800: Best error = 0.5705152943125742
Generation 900: Best error = 0.5705152943125742
Best solution: [Fraction(-139, 90), Fraction(-288, 73), Fraction(1, 3), Fraction(-13, 49)]
Final error: 0.5705152943125742

Final solution:
x = -139/90
y = -288/73
z = 1/3
t = -13/49

Verifying solution in equations:
Equation 1: 2.9996928939003293 ≈ 3, error: 0.00030710609967066205
Equation 2: 17.003129562327214 ≈ 17, error: 0.0031295623272136197
Equation 3: 17.000338582921753 ≈ 17, error: 0.0003385829217528169
Equation 4: -8.433041344391636 ≈ -9, error: 0.566958655608